In [1]:
from Alpaca_Scraper import Alpaca_Scraper
from Company_Lister import Company_Lister

import glob
import re
import pickle

import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

We can do beta testing with the google stock. This should scale to other stocks. This cell updates the stock data we store for the stocks we are interested in.

In [2]:
data_loc = '../data_files/stock_data/'
tickers = Company_Lister().get_snp_since(datetime.datetime(2011, 1, 1))
yesterday_str = str((datetime.datetime.now() - relativedelta(days=3)).date())
Alpaca_Scraper().update_tickers_data(tickers, '2011-01-01', yesterday_str)

KeyError: 'timestamp'

Here we read the stock data in for analysis

In [3]:
csvs = glob.glob(data_loc + '*.csv')
stock_dfs = {}
for csv in csvs:
    symbol = re.search(r'/((\w|\.)+)\.csv', csv).groups()[0]
    stock_df = pd.read_csv(csv)
    stock_df['timestamp'] = pd.to_datetime(stock_df['timestamp'])
    stock_df['date'] = stock_df['timestamp'].dt.date
    stock_df = stock_df.drop(stock_df.loc[stock_df['date'].duplicated()].index)
    stock_df = stock_df.drop(columns='date', axis=1)
    stock_dfs[symbol] = stock_df

### To estimate the change in stock price after a week, given 8 weeks of data, will will format the input as 8 weeks, or 40 days of stock market data where every input begins on a Monday, and ends on a Friday. That is, each week must be composed of 5 days. If there is a missing value, it will be imputed by the mean of its neighbors. Each of the daily inputs will have
* Closing Price
* Volume

**The input as a whole will be tied to 12 indicator variables that capture the month of the year**

### We will start a new input, response pair for every week of stock price data. This creates a rough estimate of 5 years * 52 weeks ~ 250 records


In [4]:
# Returns the timestamp of the first monday in the dataset
# This function assumes
def get_first_monday(stock_df):
    #Add weekday (as an index starting with 0 as sunday) to the dataset
    stock_df['weekday'] = stock_df['timestamp'].apply(lambda x: int(datetime.datetime.strftime(x, '%w')))
    starting_weekday = stock_df.iloc[0, stock_df.columns.get_loc('weekday')]
    starting_timestamp = stock_df.iloc[0, stock_df.columns.get_loc('timestamp')]
    
    #Identify the first monday in the dataset
    if starting_weekday <= 1:
        return relativedelta(days=1 - int(starting_weekday)) + starting_timestamp
    else:
        return relativedelta(days=8 - int(starting_weekday)) + starting_timestamp

In [5]:
## Converts one of the variables in the dataframe to one hot encoding
def to_one_hot(df, var, dummy=False):
    series = df[var]
    df_wo_var = df.drop(columns=var, axis=1)
    var_vals = dict([(v, i) for i, v in enumerate(list(series.unique()))])
    rev_var = dict([(i, v) for v, i in var_vals.items()])
    var_ind = series.apply(lambda x: var_vals[x])
    new_mat = var_ind.values[:, np.newaxis] == np.arange(len(rev_var))[np.newaxis, :]
    columns = [rev_var[i] for i in range(len(rev_var))]
    new_df = pd.DataFrame(new_mat, columns=columns).astype(int)
    if dummy:
        new_df.drop(columns=rev_var[len(rev_var) - 1], axis=1)
    return pd.concat((df_wo_var, new_df), axis=1)

In [6]:
# Assigns every row in the dataframe an index that corresponds to the week it belongs to.
# This index is computed from the first monday in the dataframe
def assign_week_index(stock_df):
    #Identify the first monday in the dataset
    first_time = get_first_monday(stock_df)
        
    #Find each monday to the end of the dataset. Create a dictionary matching the monday
    #with an index since the first monday
    max_date = stock_df['timestamp'].max()
    mondays = [first_time.date()]
    while True:
        cur_len = len(mondays)
        next_date = first_time + relativedelta(days = cur_len * 7)
        if next_date > max_date:
            break
        mondays.append(next_date.date())
    
    # Assign each day in the dataset to a week index. To be assigned to a week index,
    # date() >= mondays[i] & date() < mondays[i + 1]
    week_assignments = []
    cur_week = 0
    for i, row in stock_df.iterrows():
        row_date = row['timestamp'].date()
        if row_date < mondays[cur_week]:
            week_assignments.append(np.nan)
            continue
        j = 0
        while row_date >= mondays[cur_week + j]:
            j += 1
            if cur_week + j == len(mondays):
                break
        week_assignments.append(cur_week + j - 1)

        cur_week += (j - 1)
    stock_df['week_index'] = week_assignments
    return stock_df, first_time

In [7]:
## Generates all combinations of weekday and week index. Uses the date of the first monday when the dataset
## starts to compute a week number for each of the values in case the week number is missing in the actual dataset
def get_index_day_combs(first_time, num_mondays):
    tups = []
    cur_day = first_time
    for i in range(num_mondays):
        for j in range(1, 6):
            tups.append((i, j, datetime.datetime.strftime(cur_day, '%b')))
            cur_day = cur_day + relativedelta(days = 1)
        cur_day = cur_day + relativedelta(days = 2) # Get to the next monday
    all_df = pd.DataFrame(tups, columns=['week_index', 'weekday', 'start_time'])
    return all_df

In [8]:
## Imputes the given columns of the dataframe with the average of the most recent previous complete value
## and the next complete value

# Function assumes that if one of impute_cols has a missing value, all will have missing values
def impute_with_neighbors(stock_df, impute_cols):
    prev_close = len(stock_df.loc[~pd.isna(stock_df['close'])])
    prev_non_missing = np.nan
    col_locs = [stock_df.columns.get_loc(col) for col in impute_cols]
    cur_loc = 0
    for i, row in stock_df.iterrows():
        average_of = []
        if not pd.isna(stock_df.iloc[cur_loc, col_locs[0]]):
            prev_non_missing = cur_loc
            cur_loc += 1
            continue
        if not pd.isna(prev_non_missing):
            average_of.append(prev_non_missing)
        j = 1
        while cur_loc + j < len(stock_df) and pd.isna(stock_df.iloc[cur_loc + j, col_locs[0]]):
            j += 1
        if cur_loc + j != len(stock_df):
            average_of.append(cur_loc + j)
        if len(average_of) == 0:
            raise ValueError('empty dataset')
        # Actully insert the new values
        for col_loc in col_locs:
            stock_df.iloc[cur_loc, col_loc] = stock_df.iloc[average_of, col_loc].mean()
        cur_loc += 1
    new_close = len(stock_df.loc[~pd.isna(stock_df['close'])])
    return stock_df

In [28]:
## Create training examples with each input containig 8 weeks worth of closing price and volume,
## along with 12 indicator variables indicating the month of the year. The response variable
## will be the percent change in price over the following week. This will be measured as closing
## price from the last day in the data set (friday) to closing price on the following friday.
## A training example will be created from every week in the dataset

def create_examples(imputed_df, input_cols):
    input_weeks = 8
    change_after = 1
    max_week = imputed_df['week_index'].max()
    imputed_df = imputed_df.sort_values(['week_index', 'weekday'])
    start_time_col = imputed_df.columns.get_loc('start_time')
    
    inputs = []
    if max_week < 100:
        raise ValueError("Not Enough Examples")
    queries = 0
    stacking = 0
    cleaning = 0
    
    for i in range(max_week - (input_weeks - 1 + change_after)):
        st = datetime.datetime.now()
        input_time = imputed_df.loc[imputed_df['week_index'] == i].iloc[0, start_time_col]
        target_day = imputed_df.loc[(imputed_df['week_index'] == i + (input_weeks - 1) + change_after) & (imputed_df['weekday'] == 5)]
        last_day = imputed_df.loc[(imputed_df['week_index'] == i + (input_weeks - 1)) & (imputed_df['weekday'] == 5)]
        response = target_day.iloc[0, target_day.columns.get_loc('close')] / last_day.iloc[0, last_day.columns.get_loc('close')]
        input_df = imputed_df.loc[(imputed_df['week_index'] >= i) & (imputed_df['week_index'] < i + input_weeks)]
        en = datetime.datetime.now()
        queries += (en - st).total_seconds()
        st = datetime.datetime.now()
        input_df = input_df.set_index(['week_index', 'weekday'])[input_cols]
        input_stack = input_df.stack(dropna=False)
        input_stack.index.names = ['week_index', 'weekday', 'metric']
        input_stack.name = 'value'
        input_df = input_stack.reset_index()
        input_df['index'] = input_df.apply(lambda x: 'wi' + str(x['week_index'] - i) + '_wd' + str(x['weekday']) + '_' + x['metric'], axis=1)
        input_df = input_df.set_index('index')
        day_series = input_df['value']
        en = datetime.datetime.now()
        stacking += (en - st).total_seconds()
        
        st = datetime.datetime.now()
        final_series = pd.concat([day_series, pd.Series([input_time, response], index=['start_time', 'week_change'])])
        final_series.name=i
        inputs.append(final_series)
        en = datetime.datetime.now()
        cleaning += (en - st).total_seconds()
        
    ret_df = pd.DataFrame(inputs)
    print('queries', queries)
    print('stacking', stacking)
    print('cleaning', cleaning)
    ret_df = to_one_hot(ret_df, 'start_time')
    return ret_df
            

In [29]:

# This function assumes that the dataframe is in ascending order in terms of date
# and that the frequency is 'day' - each row represents one day closing price
def produce_ind_and_response(stock_df):
    
    st = datetime.datetime.now()
    ## Assing a week index to each of the rows in the dataset
    with_week_index, first_time = assign_week_index(stock_df)
    en = datetime.datetime.now()
    wi = (en - st).total_seconds()
    
    st = datetime.datetime.now()
    ## Generate all possible combinations of week index and day of week
    all_df = get_index_day_combs(first_time, int(with_week_index['week_index'].max()))
    en = datetime.datetime.now()
    al = (en - st).total_seconds()
    
    st = datetime.datetime.now()
    ## Join the dataframes to locate missing values
    with_missing = pd.merge(all_df, stock_df, how='left', on=['week_index', 'weekday']).reset_index(drop=True)
    en = datetime.datetime.now()
    wm = (en - st).total_seconds()
    
    
    st = datetime.datetime.now()
    ## Drop the last week of data as it is often incomplete
    with_missing_truncated = with_missing.drop(with_missing.loc[with_missing['week_index'] == with_missing['week_index'].max()].index)
    en = datetime.datetime.now()
    wmt = (en - st).total_seconds()
    
    st = datetime.datetime.now()
    ## Impute missing values with the mean of the last non-null value and the next non-null value
    imputed_df = impute_with_neighbors(with_missing_truncated, ['close', 'volume'])
    en = datetime.datetime.now()
    idf = (en - st).total_seconds()
    
    st = datetime.datetime.now()
    ## Create the actual training examples
    formatted_df = create_examples(imputed_df, ['close', 'volume'])
    en = datetime.datetime.now()
    fdf = (en - st).total_seconds()
                        
    print('week index', wi)
    print('all_df', al)
    print('with_missing', wm)
    print('truncated', wmt)
    print('imputed', idf)
    print('formatted', fdf)
    return formatted_df
    
        
    
    

In [30]:
df = produce_ind_and_response(stock_dfs['GOOGL'])

queries 0.6334279999999989
stacking 1.7217699999999996
cleaning 0.10707300000000011
week index 0.112905
all_df 0.048827
with_missing 0.003341
truncated 0.001045
imputed 0.172918
formatted 2.505165


In [ ]:
## Returns a number of metrics given predicted and actual values
def get_metrics(predicted, actual):
    # MSE
    np_pred = np.array(predicted)
    np_act = np.array(actual)
    mse = ((np_pred - np_act) ** 2).mean()
    acc = (np.maximum(0, 1 - np.absolute(np_pred - np_act) / np_act)).mean()
    hl_acc = (((np_pred > 1) & (np_act > 1)) | ((np_pred < 1) & (np_act < 1))).mean()
    return {'Mean Squared': mse, 'Accuracy': acc, 'High Low Accuracy': hl_acc}

In [ ]:
## Display Training and testing metrics using the get_metrics function above
def display_metrics(train_y_pred, train_act, test_y_pred, test_act):
    met_dic = get_metrics(train_y_pred, train_act)
    keys = []
    items = []
    for key, item in met_dic.items():
        keys.append(key)
        items.append(items)
    train_series = pd.Series(items, index=keys, name='Train')
    
    met_dic = get_metrics(test_y_pred, test_act)
    keys = []
    items = []
    for key, item in met_dic.items():
        keys.append(key)
        items.append(items)
    test_series = pd.Series(items, index=keys, name='Test')
        
    print(pd.DataFrame([train_series, test_series]))

In [ ]:
## Runs k-fold cross validation
## Assumes all variables other than the response are
## predictors
def cross_validate(k, in_df, response_var, model_instantiate):
    shuffled = in_df.sample(frac=1)
    test_ind = [int(i) for i in np.floor(np.linspace(0, len(shuffled), k + 1))]
    metrics = {}
    for i in range(k):
        train = pd.concat((in_df.iloc[0: test_ind[i]], in_df.iloc[test_ind[i + 1]:]))
        test = in_df.iloc[test_ind[i]:test_ind[i + 1]]
        train_y = train[response_var]
        train_x = train.drop(columns=[response_var], axis=1).copy()
        test_y = test[response_var]
        test_x = test.drop(columns=[response_var], axis=1).copy()
        
        model = eval(model_instantiate)
        model.fit(train_x, train_y)
        tr_pred = model.predict(train_x)
        te_pred = model.predict(test_x)
        
        metric_dic = get_metrics(tr_pred, train_y)
        for key, value in metric_dic.items():
            if key not in metrics:
                metrics[key] = [0, 0]
            metrics[key][0] += value
            
        metric_dic = get_metrics(te_pred, test_y)
        for key, value in metric_dic.items():
            metrics[key][1] += value
    df = pd.DataFrame(metrics, index=pd.Series(['Train', 'Test'], name='Phase'))
    df = df.transform(lambda x: x / k)
    return df       
        
        

In [ ]:
## Performs validation on one fold of the testing set and graphs the results
def hl_acc_visual(in_df, response_var, model_instantiate):
    y = in_df[response_var]
    x = in_df.drop(columns=response_var, axis=1)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, shuffle=True)
    model = eval(model_instantiate)
    model.fit(train_x, train_y)
    pred = model.predict(test_x)
    hl = ((pred > 1) & (test_y > 1)) | ((pred < 1) & (test_y < 1))
    bins = pd.cut(pred, 5)
    return hl.groupby(bins).mean()
    

In [ ]:
# Trains Random Forest Regression Models
dfs = []
for symbol, stock_df in stock_dfs.items():
    print('processing', symbol)
    try:
        res = produce_ind_and_response(stock_df)
    except ValueError:
        continue
    cv = cross_validate(10, res, 'week_change', "RandomForestRegressor(n_estimators=20)")
    cv['Symbol'] = symbol
    dfs.append(cv)

results = pd.concat(dfs, axis=0)

    

In [134]:
produce_ind_and_response(stock_dfs['BBWI'])
#stock_dfs['CTRA']

ValueError: Not Enough Examples

In [ ]:


print('Explainable Boosting Regressor')
cross_validate(10, res, 'week_change', "ExplainableBoostingRegressor()")
print('Linear Regression')
cross_validate(10, res, 'week_change', "LinearRegression()")
print('Random Forest')
cross_validate(10, res, 'week_change', "RandomForestRegressor(n_estimators=20)")
print(hl_acc_visual(res, 'week_change', "RandomForestRegressor(n_estimators=20)"))

In [ ]:
s = pd.Series(['a', 'b', 'c'], index=[1, 1, 2], name='alpha')
t = pd.Series(['a', 'b', 'c'], index=[1, 1, 2], name='alpha')

df = pd.concat((s, t), axis=1)
df['alpha'] = 1

df